# Лабораторная работа №1
## "PySpark SQL"
### вариант 1
### Лукин А.В.

Напишите программу на Python согласно вашему варианту для датасета “brooklyn_sales_map.csv”. Напишите небольшой отчёт, который должен содержать титульный лист, вариант задания, код программы и полученные вами результаты. Для помощи в выполнении задания можно воспользоваться книгами «Изучаем Spark. Молниеносный анализ данных -2015», «PySpark SQL Recipes» или интернетом.

В начале используем чёрную магию, чтобы установить Apache Spark.
После, приступаем к работе.
Из pyspark.sql импортируем SparkSession для работы с таблицами.
* builder - атрибут класса, для создания экземпляров SparkSession
* master - задает главный URL-адрес Spark для подключения, например «local» для локального запуска, «local[4]» для локального запуска с 4 ядрами или «spark://master:7077» для запуска в автономном кластере Spark.
* getOrCreate - получает существующий SparkSession или, если существующего нет, создает новый на основе параметров, установленных в этом билдере.

Для отображения датафрейма, без "show", используется конфигурация: spark.conf.set('spark.sql.repl.eagerEval.enabled', True).

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

Импортируем, необходимые библиоткеи

In [2]:
from pyspark.sql.functions import isnan, when, count, col, isnull, rand
import pyspark.sql.functions as f

Прочитаем документ: brooklyn_sales_map.csv.
* header - заголовок
* inferSchema - параметр, для определения типа данных в столбцах (str, int, double)

In [3]:
df = spark.read.csv('brooklyn_sales_map.csv', header=True, inferSchema=True)
df

_c0,borough1,neighborhood,building_class_category,tax_class,block,lot,easement,building_class,address9,apartment_number,zip_code,residential_units,commercial_units,total_units,land_sqft,gross_sqft,year_built,tax_class_at_sale,building_class_at_sale,sale_price,sale_date,year_of_sale,Borough23,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCent,HealthArea,SanitBoro,SanitDistr,SanitSub,Address37,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptLand,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area
1,3,DOWNTOWN-METROTECH,28 COMMERCIAL CO...,4,140,1001,null,R5,330 JAY STREET,COURT,11201,0,1,1,0.0,0.0,2002,4,R5,4.99401179E8,2008-04-23,2008,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,3,DOWNTOWN-FULTON F...,29 COMMERCIAL GA...,4,54,1,null,G7,85 JAY STREET,null,11201,0,0,0,134988.0,0.0,0,4,G7,3.45E8,2016-12-20,2016,BK,302,21,3017,13,33,11201,L118,84,36,1000,3,02,1B,87 JAY STREET,M1-2/R8,NA,NA,NA,NA,NA,MX-2,NA,NA,NA,N,G7,10,0,P,85 JAY STREET BRO...,134988,0,0,0,0,0,0,0,0,0,7,0,0,0,0,490.5,275.33,0,0,NA,0,N,5,5,1571850,1571850,1571850,1571850,0,0,0,NA,NA,0,7.2,2,6.5,3,3000540001,0,0021,988208,195011,12d,NA,302 016,30101,NA,3000540001,12/06/2002,1,NA,NA,17V1.1,0,1559.88914353,140131.577176
3,3,BROOKLYN HEIGHTS,21 OFFICE BUILDINGS,4,204,1,null,O6,29 COLUMBIA HEIGHTS,null,11201,0,0,0,32000.0,304650.0,1924,4,O6,3.4E8,2016-08-03,2016,BK,302,1,1003,13,33,11201,E205,84,38,2300,3,02,1A,29 COLUMBIA HEIGHTS,M2-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,N,O6,05,0,NA,25-30 COLUMBIA HE...,32000,304650,304650,0,264750,0,0,0,0,39900,2,1,13,0,0,92.42,335.92,335,92,NA,0,Y,3,5,1548000,25463250,0,0,1924,1980,0,NA,NA,9.52,0,2,0,3,3002040001,0,0001,985952,195007,12d,NA,302 004,30106,NA,0,NA,1,NA,NA,17V1.1,0,890.718521021,34656.4472405
4,3,MILL BASIN,22 STORE BUILDINGS,4,8470,55,null,K6,5120 AVENUE U,null,11234,0,123,123,905000.0,2548000.0,1970,4,K6,2.76947E8,2012-11-28,2012,BK,318,698,2005,22,46,11234,E323,63,35,8822,3,18,4E,5120 AVENUE U,M3-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,N,K6,05,1,NA,BROOKLYN KINGS PLAZA,905000,2548000,2548000,0,0,1263000,1285000,0,0,0,2,4,2,0,123,930,859,0,0,NA,0,Y,3,5,36532350,149792400,0,0,1970,0,0,NA,NA,2.82,0,2,0,3,3084700055,0,0698,1006597,161424,23b,NA,319 077,32502,NA,0,NA,1,1,1,17V1.1,0,3729.78685686,797554.847834
5,3,BROOKLYN HEIGHTS,26 OTHER HOTELS,4,230,1,null,H8,21 CLARK STREET,null,11201,0,1,1,20267.0,356000.0,1927,4,H8,2.025E8,2017-10-31,2017,BK,302,3.01,2000,13,33,11201,E205,84,38,2300,3,02,1A,79 WILLOW STREET,R6,NA,NA,NA,NA,NA,NA,NA,NA,LH-1,N,H8,08,0,X,WATCHTOWER C/O RE...,20267,356000,30000,0,30000,0,0,0,0,0,2,1,15,0,1,202,100.33,202,100.33,NA,0,N,3,2,2250000,27166050,2250000,27166050,1927,1998,2000,Brooklyn Heights ...,NA,17.57,2.43,0,4.8,3,3002300001,0,000301,985622,193713,12d,NA,302 014,30106,NA,0,NA,1,NA,NA,17V1.1,0,620.761169374,21360.1476315
6,3,WILLIAMSBURG-SOUTH,31 COMMERCIAL VA...,4,2428,1,null,V1,329 KENT AVENUE,null,11249,0,0,0,57600.0,0.0,0,4,V1,1.85445E8,2012-10-11,2012,BK,301,551,1009,14,33,11249,E221,90,30,400,3,01,2B,325 KENT AVENUE,R6,NA,NA,NA,C2-4,NA,NA,NA,NA,NA,N,D7,04,0,NA,325 KENT LLC,57600,382746,11200,371546,1822,9378,0,0,0,0,2,1,0,522,525,180,320,179,315,NA,0,N,3,5,3110400,5579990,0,0,2015,0,0,NA,NA,6.64,2.43,0,4

Краткое описание файла

In [4]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- borough1: integer (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- building_class_category: string (nullable = true)
 |-- tax_class: string (nullable = true)
 |-- block: integer (nullable = true)
 |-- lot: integer (nullable = true)
 |-- easement: string (nullable = true)
 |-- building_class: string (nullable = true)
 |-- address9: string (nullable = true)
 |-- apartment_number: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- residential_units: integer (nullable = true)
 |-- commercial_units: integer (nullable = true)
 |-- total_units: integer (nullable = true)
 |-- land_sqft: double (nullable = true)
 |-- gross_sqft: double (nullable = true)
 |-- year_built: integer (nullable = true)
 |-- tax_class_at_sale: integer (nullable = true)
 |-- building_class_at_sale: string (nullable = true)
 |-- sale_price: double (nullable = true)
 |-- sale_date: string (nullable = true)
 |-- year_of_sale: integer (nulla

In [5]:
df.count(), len(df.columns)

(390883, 111)

## 1. Найдите среднюю стоимость жилья (sale_price) и выведите новую таблицу, содержащую две колонки – стоимость жилья и отклонение стоимости от среднего значения.

Посмотрим сколько объектов с низкой ценой.

In [6]:
df.groupby("sale_price").agg({"sale_price":"count"}).\
    orderBy("sale_price", ascending=True).\
    select("sale_price", col("count(sale_price)").alias("count_sp")).show(11)

+----------+--------+
|sale_price|count_sp|
+----------+--------+
|       0.0|  140143|
|       1.0|     709|
|       2.0|      17|
|       3.0|      15|
|       4.0|      12|
|       5.0|      23|
|       6.0|       9|
|       7.0|      19|
|       8.0|      12|
|       9.0|      15|
|      10.0|    3465|
+----------+--------+
only showing top 11 rows



Для **среднего значения sale_price**, будем считать sale_price > **0**
*(уточнить от какой стоимости лучше брать)*

In [7]:
mean_sp = df.select("sale_price").filter("sale_price>0").agg({"sale_price": "mean"}).\
             select(col("avg(sale_price)").alias("mean_sp"))
mean_sp

mean_sp
789988.476126665


In [8]:
diff_mean_sp = df.select("sale_price").filter("sale_price>0").\
               withColumn("diff_mean_sp", f.round(df.sale_price-mean_sp.first()[0], 2))
diff_mean_sp.orderBy(rand()).show(5)

+----------+------------+
|sale_price|diff_mean_sp|
+----------+------------+
|  395000.0|  -394988.48|
| 1500000.0|   710011.52|
|  300000.0|  -489988.48|
|  420000.0|  -369988.48|
|  422300.0|  -367688.48|
+----------+------------+
only showing top 5 rows



### В таблице представлены две колонки – стоимость жилья и отклонение стоимости от среднего значения

## 2. Найдите среднюю стоимость жилья (sale_price) для каждого района.

Как и в прошлом задании, **"sale_price"** возьмём больше **нуля**.

In [9]:
df.filter("sale_price>0").\
    groupby("neighborhood").agg({"sale_price": "avg"}).\
    select("neighborhood", f.round(col("avg(sale_price)")).alias("mean_sp")).\
    orderBy("avg(sale_price)",ascending=True).show(10)

+---------------+--------+
|   neighborhood| mean_sp|
+---------------+--------+
|           3004|340704.0|
|GERRITSEN BEACH|371206.0|
|      FLATLANDS|384301.0|
| OLD MILL BASIN|408159.0|
|  EAST NEW YORK|432243.0|
|       CANARSIE|437302.0|
|  CYPRESS HILLS|455799.0|
|    MARINE PARK|464091.0|
|     OCEAN HILL|499540.0|
|  FLATBUSH-EAST|514859.0|
+---------------+--------+
only showing top 10 rows



### В таблице представлена средняя цена для каждого района.

## 3. Выведите среднюю полную площадь жилья (gross_sqft) для всех сочетаний налоговых категорий (tax_class) и лет продажи (year_of_sale).


В этом задании площадь возьмём больше нуля, и округлим среднее значение до верхнего целого значения.

In [10]:
df.filter("gross_sqft!=0")\
    .groupby("tax_class", "year_of_sale").agg({"gross_sqft": "avg"}).\
    select("tax_class", "year_of_sale", f.ceil(col("avg(gross_sqft)")).alias("mean_gs")).\
    orderBy(rand()).show(10)

+---------+------------+-------+
|tax_class|year_of_sale|mean_gs|
+---------+------------+-------+
|       2A|        2006|   4101|
|        4|        2012|  17688|
|       2A|        2010|   4059|
|       2B|        2013|   6813|
|       2C|        2005|   7476|
|        2|        2017|  34266|
|       2C|        2007|   5023|
|       2A|        2013|   4047|
|       2C|        2015|   4590|
|       2B|        2015|   6815|
+---------+------------+-------+
only showing top 10 rows



### В таблице представлена средняя полная площадь жилья для всех сочетаний налоговых категорий и лет продажи.

## 4. Выведите таблицу, содержащую количество пустых (null) значений для каждой колонки

In [11]:
df = df.replace("NA", None)
null_count = df.select([count(when(isnull(c), c)).alias(c) for c, t in df.dtypes])
null_count.show(vertical=True)

-RECORD 0-------------------------
 _c0                     | 0      
 borough1                | 0      
 neighborhood            | 0      
 building_class_category | 83     
 tax_class               | 6934   
 block                   | 0      
 lot                     | 0      
 easement                | 390883 
 building_class          | 6934   
 address9                | 0      
 apartment_number        | 305267 
 zip_code                | 0      
 residential_units       | 0      
 commercial_units        | 0      
 total_units             | 0      
 land_sqft               | 0      
 gross_sqft              | 0      
 year_built              | 0      
 tax_class_at_sale       | 0      
 building_class_at_sale  | 0      
 sale_price              | 0      
 sale_date               | 0      
 year_of_sale            | 0      
 Borough23               | 87155  
 CD                      | 87155  
 CT2010                  | 87447  
 CB2010                  | 88368  
 SchoolDist         

### В таблице представлено кол-во пустых (null) значений для каждой колонки.

In [12]:
spark.stop()

## Вывод: я выведен